In [1]:
%matplotlib inline

In [2]:
cd /home/andre/PycharmProjects/stocks/

/home/andre/PycharmProjects/stocks


In [3]:
from matplotlib import pyplot as plt
import numpy as np
import os 
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import talib as ta
import datetime
from Tools.util import progressbar

In [4]:
from Tools import Meta5_Ibov_Load as meta5load

In [5]:
meta5filepath = "/home/andre/.wine/drive_c/Program Files/Rico MetaTrader 5/MQL5/Files"

In [6]:
os.chdir(meta5filepath)

In [7]:
meta5load.Set_Data_Path(meta5filepath, meta5filepath)

must load metatrader 5 *.mt5bin files


In [8]:
masterdf = meta5load.Load_Meta5_Data(suffix='RTM1.mt5bin', cleandays=False)

Couldn't load data: missing *RTM1.mt5bin files!


# Load Vectors Ready for Training

In [6]:
# backup things
Xo = X
Yo = Y

In [7]:
X = X.values.astype(np.float32) # due Float tensors
y = Y.values.astype(np.int64) # due Cross Entropy Loss requeires Tensor Long

In [8]:
np.unique(y)

array([0, 1])

In [9]:
X.shape

(504816, 157)

In [10]:
y.shape

(504816,)

In [11]:
batch_size = 64

In [12]:
# change here if you wish OPTIONS
# shift=60
nforecast=60
nvalidation=nforecast # to validate the model prior prediction
ntraining = 5*8*60 # 5*8 hours before for training - 1 week or 8 hours
nwindow = nvalidation+nforecast+ntraining 

In [3]:
(256*30)/(5*8*60)

3.2

In [13]:
print(ntraining)
print(batch_size)
print(nvalidation)

2400
64
60


In [14]:

#Xo.skew()

In [15]:
begin = 10000

## Simple non-parallel version for testings

In [16]:
th.cuda.get_device_name(0)

'GeForce GTX 1060'

In [17]:
import copy
from torch.optim import lr_scheduler
device = th.device("cuda" if th.cuda.is_available() else "cpu")
input_size = X.shape[1]

In [18]:
def TorchModelPredict(model, X):
    y_prob = model(X)     
    y_pred = th.argmax(y_prob, 1)
    y_pred = y_pred.to("cpu")
    return y_prob, y_pred.data.numpy()


# Using Multiple threads on GPU by CUDA

### Need to work on this someday

In [19]:
def tensor_shuffle(X, y, size):
    """better use the dataset api in the future"""
    i = np.random.randint(X.shape[0]-size)
    return X[i:i+size], y[i:i+size]

def fastTrainTorchNet(X, y, X_s, y_s, input_size, model=None, nepochs=30, device="cuda", 
                  batch_size=256, verbose=True):
    """
    X, y         : vectors for training
    X_s, y_s     : vector for validation (scoring the model)
    
    the best model with smaller error in validation will be returned
    after the nepochs of training
    
    """
    if model == None:
        model = th.nn.Sequential(th.nn.Linear(input_size, 1024), th.nn.ReLU(), th.nn.Dropout(.1),
                                 th.nn.Linear(1024,280), th.nn.ReLU(), th.nn.Dropout(.3),
                                 th.nn.Linear(280, 70), th.nn.ReLU(),  th.nn.Dropout(.05),
                                 th.nn.Linear(70, 2), th.nn.Softmax(dim=1)) # dim == 1 collumns add up to 1 probability
        criterion = th.nn.CrossEntropyLoss()
        model = model.to(device)
        #weight_decay regularization not applied
        optimizer = th.optim.Adam(model.parameters(), lr=0.1) # learning rate optimal
    # Decay LR by a factor of 0.1 every 10 epochs
    scheduler = lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    
    best_model = None
    best_lossv = 1000.
    best_error = 100.
    for t in range(nepochs):
        X_t, y_t = tensor_shuffle(X, y, batch_size)        
        y_pred = model(X_t)         
        lossy = criterion(y_pred, y_t)
        optimizer.zero_grad()
        lossy.backward()
        optimizer.step()
        model.eval()
        y_pred = th.argmax(y_pred, 1)         # calculate error complement of accuracy
        error_train = th.nn.functional.mse_loss(
            y_pred.float(), y_t.float())
        y_pred = model(X_s)          # on validation set
        lossv = criterion(y_pred, y_s)
        y_pred = th.argmax(y_pred, 1)
        error_validation = th.nn.functional.mse_loss(
            y_pred.float(), y_s.float())
        if lossv < best_lossv: # now loss for validation is smaller
            best_lossv = lossv     # lets save this network
            best_errorv = error_validation
            best_errort = error_train
            best_model = copy.deepcopy(model.state_dict()) # save the actual weights
        
        if verbose:
            if t%int(nepochs/5) == 0: 
                # on the loss values could apply a kind of normalization due the fact that they 
                # don't have same size of samples BUT just BEAR ON MIND
                # loss error on validation set absolute value has NOTHING to do with
                # with the absolute value of the training set
                print("iteration : {:6d} l training: {:.2f} l validation: {:.2f}"
                      " Err trainging :{:.2f}  Err validation ; {:.3f}".format(
                      t, lossy, lossv, error_train, error_validation))

        model.train()                
        scheduler.step()

    # return the best model in the validation and the accuracy
    best_errorv = best_errorv.to("cpu").item()
    best_errort = best_errort.to("cpu").item()
    model.load_state_dict(best_model)
    #print(best_errort, best_errorv)
    # return the model and accuracy of training and validation
    return model, best_model, best_errort, best_errorv

In [20]:
def tensor_shuffle(X, y, size):
    """better use the dataset api in the future"""
    i = np.random.randint(X.shape[0]-size)
    return X[i:i+size], y[i:i+size]

def TrainTorchNet(X, y, X_s, y_s, input_size, model=None, nepochs=30, device="cuda", 
                  batch_size=256, verbose=True):
    """
    X, y         : vectors for training
    X_s, y_s     : vector for validation (scoring the model)
    
    the best model with smaller error in validation will be returned
    after the nepochs of training
    
    """
    if model == None:
        model = th.nn.Sequential(th.nn.Linear(input_size, 1024), th.nn.ReLU(), th.nn.Dropout(.1),
                                 th.nn.Linear(1024,280), th.nn.ReLU(), th.nn.Dropout(.3),
                                 th.nn.Linear(280, 70), th.nn.ReLU(),  th.nn.Dropout(.05),
                                 th.nn.Linear(70, 2), th.nn.Softmax(dim=1)) # dim == 1 collumns add up to 1 probability
        criterion = th.nn.CrossEntropyLoss()
        model = model.to(device)

        #weight_decay regularization not applied
        optimizer = th.optim.Adam(model.parameters(), lr=0.1) # learning rate optimal
        
        #optimizer.to(device)

    # Decay LR by a factor of 0.1 every 10 epochs
    scheduler = lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    
    best_model = None
    best_lossv = 1000.
    best_error = 100.

    for t in range(nepochs):
        
        X_t, y_t = tensor_shuffle(X, y, batch_size)        
        
        y_pred = model(X_t)         
        # Compute and print loss
        lossy = criterion(y_pred, y_t)
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        lossy.backward()
        optimizer.step()

        # could also store best score and try
        # to continue training from that like tensorflow checkpoints        
        # Model Evaluation
        model.eval()
        # calculate error complement of accuracy
        # on training set
        y_pred = th.argmax(y_pred, 1)
        error_train = th.nn.functional.mse_loss(
            y_pred.float(), y_t.float())

        # calculate percentage accuracy
        # on validation set
        y_pred = model(X_s) 
        lossv = criterion(y_pred, y_s)
        y_pred = th.argmax(y_pred, 1)
        error_validation = th.nn.functional.mse_loss(
            y_pred.float(), y_s.float())
        
        ### error on validation set next 60 samples / minutes
        ### todo: better tecniques of early stopping, early stopping
        ### is a regularization so other techinques might be better?
        ### or easier?
        if lossv < best_lossv: # now loss for validation is smaller
            # lets save this network
            best_lossv = lossv
            best_errorv = error_validation
            best_errort = error_train
            best_model = copy.deepcopy(model.state_dict()) # save the actual weights
        
        if verbose:
            if t%int(nepochs/5) == 0: 
                # on the loss values could apply a kind of normalization due the fact that they 
                # don't have same size of samples BUT just BEAR ON MIND
                # loss error on validation set absolute value has NOTHING to do with
                # with the absolute value of the training set
                print("iteration : {:6d} l training: {:.2f} l validation: {:.2f}"
                      " Err trainging :{:.2f}  Err validation ; {:.3f}".format(
                      t, lossy, lossv, error_train, error_validation))

        model.train()                
        scheduler.step()

    # return the best model in the validation and the accuracy
    best_errorv = best_errorv.to("cpu").item()
    best_errort = best_errort.to("cpu").item()
    model.load_state_dict(best_model)
    #print(best_errort, best_errorv)
    # return the model and accuracy of training and validation
    return model, best_errort, best_errorv

In [21]:
""""Executor of prediction"""
import numpy as np
import pandas as pd
import scipy
import sys

# change here if you wish OPTIONS
# shift=60
nforecast=60
nvalidation=nforecast # to validate the model prior prediction
ntraining = 5*8*60 # 8 hours before for training
nwindow = nvalidation+nforecast+ntraining 


def Slide_Predictions(X, y, index, input_size, verbose=False, device='cuda'):
    """
    Backtesting:
    Make predictions with a sliding window.    
    """
    size = len(X)
    # number of possible predictions with data size : size
    # because forecast are allways array of forecasts
    # defined by the shift made
    nshifts = size-(ntraining+nvalidation+nforecast)     
    
    if nshifts <= 0:
        print('somethings is wrong, array of data too small')
        print('minimum size is training+validation+forecast for 1 prediction')
        return
    
    if verbose:
        print('maximum number of predictions is: ', nshifts)    
    
    # sliding window with step of one sample shift   
    prediction_book = pd.DataFrame(index=np.arange(nshifts), columns=['tindex', 'buy', 'score'])
    
    # all to cuda memory
    X = th.tensor(X).to(device)
    y = th.tensor(y).to(device)
    
    for i in progressbar(range(nshifts)):  
        
        index_ = index[i:i+nwindow].copy()        
        # training samples are the first ones
        X_t = X[i:i+ntraining]
        y_t = y[i:i+ntraining]        

        # control samples for scoring (validation of the model)
        # SCORE using the nforecast samples just after the window       
        X_s = X[i+ntraining:i+ntraining+nvalidation]
        y_s = y[i+ntraining:i+ntraining+nvalidation]

        # predict on the last nforecast samples
        X_p = X[i+ntraining+nvalidation:i+ntraining+nvalidation+nforecast]        

        # return mode, accuracy 
        clfmodel, errort, errorv = TrainTorchNet(X_t, y_t, X_s, y_s, 
                                                       input_size, device=device, verbose=False)

        # accurary of the model if higher than 90% we can predict
        score = errorv           
        
        if verbose:
            print(i, " errort : ", errort, " errorv: ",errorv)
        
        # we cannot predict unless the model is 90%+ accurate
        # accurate on validation. on training we accept 70%+
        if errorv > 0.1 or errort > 0.3: 
            del clfmodel
            continue    

        probability, prediction = TorchModelPredict(clfmodel, X_p)        
        
        down = abs(prediction.sum()-len(prediction))/len(prediction)
        up = abs(1-down)

        if up > 0.9 or down > 0.9: # only if certain by 90% that will go up or down
            # where will it be bought in time
            # allways +5 minutes after prediction
            tindex = index_[-nforecast+5]
            buy = up > down
            if buy: # buy
                buy = 1
            else: # sell
                buy = -1
            prediction_book.iloc[i] = [tindex, buy, score]
            #print(prediction_book.iloc[i])            
            
        del clfmodel
        
    return prediction_book

# splitting data to guarantee overlapping multiprocessing
# split data in a batch with limits [start:end]
def paralel_Slide_Predictions_Up_Down(Xtrain, ytrain, sIndex, start, end):    
    X = Xtrain[start:end]
    y = ytrain[start:end]
    input_size = Xtrain.shape[1]
    index = sIndex[start:end]        
    prediction_book = Slide_Predictions(X.copy(), y.copy(), index.copy(), input_size)    
    # save recommended orders from starti to endi
    prediction_book.dropna(inplace=True) # due preallocated size there are nan samples
    return prediction_book
    
def paralel_batch_slices(nprocesses, datasize, nwindow):
    """
    create batch slice indexes for paralel processing
    should write test function for this
    altough it is perfectly working
    """
    batch_size = int(np.floor((datasize-nwindow)/nprocesses))
    nmiss = (datasize-nwindow-batch_size*nprocesses) # samples to complete the non integer division
    # last batch will have more sample to complete the non integer division
    slices = np.array([i*batch_size for i in range(nprocesses)])
    # each batch must have the previous sample size of training + forecast window
    istarts = slices
    iends = slices[1:] + nwindow
    iends = np.append(iends, datasize)
    # first and last batch have different size of samples
    print('batch starts ', istarts)
    print('batch ends ', iends)
    print('batch sizes ', iends-istarts) 
    batches = zip(istarts, iends)
    return batches

In [30]:
limit = len(X)

In [31]:
limit

504816

In [32]:
%%time
prediction_book = Slide_Predictions(X[:limit], y[:limit], Yo.index[:limit], input_size, verbose=False)

[################################################################################] 502296/502296
CPU times: user 9h 24min 6s, sys: 7min 48s, total: 9h 31min 54s
Wall time: 9h 30min 59s


In [33]:
prediction_book.dropna(inplace=True)

In [34]:
len(prediction_book)/limit

0.05318769611105829

In [35]:
prediction_book

,tindex,buy,score
20,2013-07-11 10:31:00,1,0
21,2013-07-11 10:32:00,1,0
33,2013-07-11 10:44:00,1,0
39,2013-07-11 10:50:00,1,0
43,2013-07-11 10:54:00,1,0
45,2013-07-11 10:56:00,1,0
127,2013-07-11 12:24:00,1,0.0833333
131,2013-07-11 12:28:00,1,0.0333333
140,2013-07-11 12:37:00,1,0.0333333
151,2013-07-11 12:53:00,1,0.0333333


In [36]:
prediction_book.to_pickle('allyears_PETR4_prediction_CUDA_NN_EMA120min_shift120_RFy_90_70_percent_pytorch.pickle')

## Need to reinstall CUDA so multiprocessing GPU works

###  Using multiprocessing for Pytorch + CUDA
Joblib wont work!  

In [46]:
import torch.multiprocessing as mp
from multiprocessing import set_start_method

if __name__ == '__main__':
    try:
        set_start_method('spawn')
    except RuntimeError:
        print("is not working")
        pass
    num_processes = 2
    processes = []
    batches = paralel_batch_slices(num_processes, len(X), nwindow)
    for start, end in batches:
        p = mp.Process(target=paralel_Slide_Predictions_Up_Down,
                       args=(X, y, yor.index, start, end))
        p.start()        
        processes.append(p)
    for p in processes:
        p.join()

is not working
batch starts  [     0 251178]
batch ends  [253698 504876]
batch sizes  [253698 253698]


Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/andre/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/andre/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/andre/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/andre/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-45-f9b3fa76b674>", line 100, in paralel_Slide_Predictions_Up_Down
    prediction_book = Slide_Predictions(X.copy(), y.copy(), index.copy(), input_size)
  File "<ipython-input-45-f9b3fa76b674>", line 100, in paralel_Slide_Predictions_Up_Down
    prediction_book = Slide_Predictions(X.copy(), y.copy(), index.copy(), input_size)
  File "<ipython-input-45-f9b3fa76b674>", line 3